# Creando el modelo de regresión

En este apartado cargaremos los datos históricos y crearemos un modelo de regresión para hacer los cálculos de predicción. Para este ejemplo, utilizaremos el modelo SVR (`Support Vector Regression` del módulo de [scikit-learn](https://scikit-learn.org/stable/index.html) de [Python](https://www.python.org/): [sklearn.svm.SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)), pero se podrían utilizar otros modelos de regresión para comparar los resultados.

Los pasos que se seguirán en este *notebook* son los siguientes:

 1. [Carga de datos históricos](#Carga-de-datos-históricos)
 2. [Preparación de los datos](#Preparación-de-los-datos)
 3. [Normalización](#Normalización)
 4. [Añadiendo información a los datos de entrada](#Añadiendo-información-a-los-datos-de-entrada)
 5. [Training/Test/Validation Set](#Training/Test/Validation-Set)
 6. [Creando el modelo](#Creando-el-modelo)
 7. [Visualizando los resultados](#Visualizando-los-resultados)

## Carga de datos históricos

Vamos a empezar cargando los datos:

In [3]:
import pandas as pd

df = pd.read_csv('C:/Users/alberto.irusta/Desktop/prevision-dia-tiempo-real/1. Creando el modelo predictivo/Datos/TXT_Simulación_datos_2019-01-01_2019-12-31.txt',
                 parse_dates=['ticketDate'])



df.rename(columns={"ticketDate": "Fecha", "amount": "Importe"}, inplace=True)

df = df[df['Fecha'] >= '2018-01-01']

Nuestros `dataframe` de momento sólo contiene 2 columnas:

 - `Fecha`: día y hora de la emisión del ticket (formato `%Y-%m-%d %H:%M:%S`)
 - `Importe`: importe del ticket en €

In [4]:
df.tail()

,Fecha,Importe
142779,2019-12-31 23:07:00,3665765.03
142780,2019-12-31 23:09:00,3688284.50
142781,2019-12-31 23:11:00,3710922.43
142782,2019-12-31 23:15:00,3756555.76
142783,2019-12-31 23:48:00,4151740.92


y el tipo de dato que contiene cada columna:
 - `Fecha`: tipo de dato `datetime`
 - `Importe`: tipo de dato `float`

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142784 entries, 0 to 142783
Data columns (total 2 columns):
Fecha      142784 non-null datetime64[ns]
Importe    142784 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.3 MB


## Preparación de los datos

Como se ha indicado al inicio, el objetivo es crear un modelo de **regresión**, por lo que tenemos que preparar los datos de forma que tengamos unos datos de entrada $X$ y otros de salida $Y$.

Para ello, vamos a intentar transformar nuestro `dataframe` para que tenga un formato similar a la siguiente tabla (en donde las ventas se van acumulando cada 15 minutos):

| Dia | 09:00 | 09:15 | 09:30 | ... | 21:30 | 21:45 | 22:00 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 2019-09-26 | 0 | 20 000 | 51 000 | ... | 5 000 000 | 5 000 100 | 5 000 150 |
| 2019-09-27 | 0 | 20 200 | 51 400 | ... | 5 500 000 | 5 500 100 | 5 500 150 |
| 2019-09-28 | 0 | 23 000 | 53 000 | ... | 6 000 000 | 6 000 100 | 6 000 150 |

De esta manera, tendríamos los datos de entrada $X$ (en el que se introducirán más variables):

| 09:00 | 09:15 | 09:30 | ... | 21:30 | 21:45 |
| --- | --- | --- | --- | --- | --- |
| 0 | 20 000 | 51 000 | ... | 5 000 000 | 5 000 100 |
| 0 | 20 200 | 51 400 | ... | 5 500 000 | 5 500 100 |
| 0 | 23 000 | 53 000 | ... | 6 000 000 | 6 000 100 |

Y los datos de salida $Y$:

| 22:00 |
| --- |
| 5 000 150 |
| 5 500 150 |
| 6 000 150 |

Este es un ejemplo ilustrativo, ya que se contabilizan compras fueras del rango usual laboral (compras online u horarios especiales).

Para preparar esa tabla, vamos a empezar agregando las ventas en intervalos de 15 minutos:

In [6]:
df.index = df.pop('Fecha')
df = df.resample('15T').sum()

df.tail()

,Importe
Fecha,
2019-12-31 22:45:00,3061963.21
2019-12-31 23:00:00,18340542.26
2019-12-31 23:15:00,3756555.76
2019-12-31 23:30:00,0.00
2019-12-31 23:45:00,4151740.92


Continuaremos desglosando la `Fecha` en 2 nuevas columnas:
 - `Dia`
 - `Hora`

In [7]:
df['Dia'] = df.index.map(lambda x: x.strftime('%Y-%m-%d'))
df['Hora'] = df.index.map(lambda x: x.strftime('%H:%M'))

df.tail()

,Importe,Dia,Hora
Fecha,,,
2019-12-31 22:45:00,3061963.21,2019-12-31,22:45
2019-12-31 23:00:00,18340542.26,2019-12-31,23:00
2019-12-31 23:15:00,3756555.76,2019-12-31,23:15
2019-12-31 23:30:00,0.00,2019-12-31,23:30
2019-12-31 23:45:00,4151740.92,2019-12-31,23:45


Ahora ya estamos listos para formatear nuestro `dataframe`, para ello utilizaremos la función [crosstab](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) del módulo de [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html):

In [8]:
df = pd.crosstab(index=df['Dia'],
                 columns=[df['Hora']],
                 values=df.Importe,
                 aggfunc=sum).fillna(0).reset_index()

df.set_index('Dia', inplace=True)

df.tail()

Hora,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
Dia,,,,,,,,,,,,,,,,,,,,,
2019-12-27,0.0,0.0,0.0,22.22,0.00,20.66,24.61,0.00,0.00,0.00,...,14195449.59,18674753.26,8830466.20,6993437.32,12271717.33,12893667.68,3102882.89,6234078.90,6682340.35,3468545.16
2019-12-28,0.0,0.0,0.0,0.00,49.14,0.00,0.00,21.09,23.44,11.83,...,9801415.28,6472283.75,6183345.05,6461520.56,10115455.06,10656593.21,4225363.76,11077258.09,6942534.29,7249797.96
2019-12-29,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,12.84,27.67,...,165822.22,172745.76,69842.53,109633.70,190308.41,120219.28,189956.15,99035.39,103569.34,55646.60
2019-12-30,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,9982709.16,10510408.86,8953719.48,11281191.27,0.00,4084556.12,4832984.90,2544430.12,5341264.15,2793031.34
2019-12-31,19.1,0.0,0.0,18.02,0.00,0.00,0.00,0.00,0.00,0.00,...,8471703.79,11232695.65,2620354.22,5640719.84,5887374.00,3061963.21,18340542.26,3756555.76,0.00,4151740.92


Para finalizar con esta parte, vamos hacer la suma acumulada de cada fila:

In [9]:
df = df.cumsum(axis=1)

df.tail()

Hora,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
Dia,,,,,,,,,,,,,,,,,,,,,
2019-12-27,0.0,0.0,0.0,22.22,22.22,42.88,67.49,67.49,67.49,67.49,...,1.464971e+08,1.651718e+08,1.740023e+08,1.809957e+08,1.932675e+08,2.061611e+08,2.092640e+08,2.154981e+08,2.221804e+08,2.256490e+08
2019-12-28,0.0,0.0,0.0,0.00,49.14,49.14,49.14,70.23,93.67,105.50,...,9.066097e+07,9.713325e+07,1.033166e+08,1.097781e+08,1.198936e+08,1.305502e+08,1.347755e+08,1.458528e+08,1.527953e+08,1.600451e+08
2019-12-29,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,12.84,40.51,...,2.015264e+06,2.188010e+06,2.257853e+06,2.367486e+06,2.557795e+06,2.678014e+06,2.867970e+06,2.967005e+06,3.070575e+06,3.126221e+06
2019-12-30,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.154936e+08,1.260040e+08,1.349578e+08,1.462390e+08,1.462390e+08,1.503235e+08,1.551565e+08,1.577009e+08,1.630422e+08,1.658352e+08
2019-12-31,19.1,19.1,19.1,37.12,37.12,37.12,37.12,37.12,37.12,37.12,...,1.262170e+08,1.374497e+08,1.400701e+08,1.457108e+08,1.515982e+08,1.546601e+08,1.730007e+08,1.767572e+08,1.767572e+08,1.809090e+08


## Normalización

Para la normalización de los datos vamos a dividir todos los campos por el máximo valor de la tabla. De esa manera, todos nuestros valores oscilarán entre 0 y 1.

In [10]:
max_value = df['23:45'].max()
df /= max_value
df.reset_index(inplace=True)

df.tail()

Hora,Dia,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
360,2019-12-27,0.000000e+00,0.000000e+00,0.000000e+00,8.143098e-08,8.143098e-08,1.571449e-07,2.473347e-07,2.473347e-07,2.473347e-07,...,0.536877,0.605315,0.637677,0.663306,0.708279,0.755531,0.766902,0.789749,0.814238,0.826949
361,2019-12-28,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.800863e-07,1.800863e-07,1.800863e-07,2.573761e-07,3.432781e-07,...,0.332251,0.355970,0.378631,0.402311,0.439381,0.478435,0.493920,0.534516,0.559958,0.586527
362,2019-12-29,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.705553e-08,...,0.007385,0.008019,0.008274,0.008676,0.009374,0.009814,0.010510,0.010873,0.011253,0.011457
363,2019-12-30,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.423257,0.461775,0.494588,0.535931,0.535931,0.550900,0.568611,0.577936,0.597511,0.607746
364,2019-12-31,6.999693e-08,6.999693e-08,6.999693e-08,1.360359e-07,1.360359e-07,1.360359e-07,1.360359e-07,1.360359e-07,1.360359e-07,...,0.462555,0.503720,0.513323,0.533995,0.555571,0.566792,0.634006,0.647773,0.647773,0.662988


## Añadiendo información a los datos de entrada

Una vez que hemos preparado los datos para poder aplicar un modelo de regresión, vamos a añadir información adicional a los datos para intentar crear un modelo más afinado. En este caso vamos a añadir información de calendario:

 - Día de la semana
 - Días festivos
 
Esas nuevas variables van a ser variables **categóricas** por lo que crearemos [variables *dummy*](https://medium.com/hugo-ferreiras-blog/dealing-with-categorical-features-in-machine-learning-1bb70f07262d) para introducirlo en el modelo.

### Días de la semana

In [11]:
df['Dia'] = pd.to_datetime(df['Dia'])
weekdays = [
    [0, 'Lunes'],
    [1, 'Martes'],
    [2, 'Miercoles'],
    [3, 'Jueves'],
    [4, 'Viernes'],
    [5, 'Sabado'],
    [6, 'Domingo']
]
for weekday, weekday_name in weekdays:
    df[weekday_name] = df['Dia'].map(lambda x: x.weekday() == weekday)
    
df[['Dia', 'Lunes', 'Martes', 'Miercoles',
    'Jueves', 'Viernes', 'Sabado', 'Domingo']].tail()

Hora,Dia,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado,Domingo
360,2019-12-27,False,False,False,False,True,False,False
361,2019-12-28,False,False,False,False,False,True,False
362,2019-12-29,False,False,False,False,False,False,True
363,2019-12-30,True,False,False,False,False,False,False
364,2019-12-31,False,True,False,False,False,False,False


### Días Festivos

In [12]:
calendario = ['2019-01-01','2019-01-01','2019-01-06','2019-03-19',
              '2019-04-28','2019-05-15','2019-07-25','2019-08-15',
              '2019-10-12','2019-11-01','2019-12-06','2019-12-08','2019-12-25','2019-12-26']

df['Festivo'] = df['Dia'].isin(calendario)

## Training/Test/Validation Set

Por último, antes de crear el modelo vamos a dividir los datos en 3 bloques:

 - **Training Set**: 80% de los datos
 - **Test Set**: 10% de los datos
 - **Validation Set**: 10% de los datos

In [13]:
def training_test_set():
    # Training Set (80%)
    train_data = df.sample(frac=0.8, random_state=0)
    test_validation_data = df.drop(train_data.index)

    # Test/Validatin Set (10%/10%)
    test_data = test_validation_data.sample(frac=0.5, random_state=0)
    validation_data = test_validation_data.drop(test_data.index)

    # Definir la variable 'Y'
    train_y = train_data.pop('23:45')
    test_y = test_data.pop('23:45')
    validation_y = validation_data.pop('23:45')

    # Eliminamos la columna 'Dia'
    train_data.drop(['Dia'], axis=1, inplace=True)
    test_data.drop(['Dia'], axis=1, inplace=True)
    validation_data.drop(['Dia'], axis=1, inplace=True)
    
    return [train_data, test_data, validation_data, train_y, test_y, validation_y]

## Creando el modelo

Como se ha indicado en la introducción, en este ejemplo se va a utilizar el modelo [SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) para la regresión y se va a elegir el `kernel` `rbf`.

No es objeto de este *notebook* explicar los detalles de este modelo, pero el lector que esté interesado en profundizar en este modelo puede ver con más detalle los algoritmos que se utilizan para hacer los cálculos en la documentación oficial de [scikit-learn](https://scikit-learn.org/stable/modules/svm.html#svm-regression).

En `SVR` hay básicamente 2 parámetros que se utilizan para ajustar el modelo:

 - `C`
 - `Epsilon`
 
a los que vamos a darle diferentes valores.

In [14]:
from sklearn.svm import SVR

def crear_modelo():
    error = -1

    for C in [0.1, 1, 100, 1000]:
        for epsilon in [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10]:
            # Creamos el modelo con los parámetros seleccionados
            svr_rbf = SVR(kernel='rbf', C=C, gamma='auto', epsilon=epsilon)

            # Ajustamos el modelo a nuestros datos
            model = svr_rbf.fit(train_data, train_y)

            # Medir la calidad del modelo con el Test Set
            error_now = (model.predict(test_data) - test_y).std()

            # Guardar los parámetros si se ha mejorado el error
            if (error_now < error) or (error == -1):
                error = error_now
                C_good = C
                epsilon_good = epsilon
                
    return SVR(kernel='rbf', C=C_good, gamma='auto', epsilon=epsilon_good)

Como queremos crear diferentes modelos para cada hora, a la hora de crear el modelo tendremos que modificar las variables de entrada $X$, eliminando las columnas que no se van a tener en cuenta para el modelo. Para ese fin vamos a definir la siguiente función:

In [15]:
def eliminar_columnas(hour_now):
    """Eliminar las columnas que no se van a utilizar para el cálculo de las previsiones"""

    cols_drop = df.columns[(df.columns > hour_now) & (df.columns < '23:45')]
    for col in cols_drop:
        df.pop(col)

También vamos a necesitar otra función para guardar los modelos utilizando el módulo `joblib`

In [16]:
import os
import joblib


def guardar_modelo_svr(name):
    """Guardar los resultados del modelo"""

    folder = 'models/'
    if not os.path.exists(folder):
        os.makedirs(folder)

    filename = 'all_data_model_' + name + '.sav'
    joblib.dump(model, folder + filename)

In [18]:
df_orig = df.copy()

Pongamos en marcha los cálculos...

In [19]:
for hour in range(10, 22):
    print('\nCreando modelo para las ' + str(hour) + '...')
    df = df_orig.copy()
    eliminar_columnas(str(hour) + ':00')
    train_data, test_data, validation_data, train_y, test_y, validation_y = training_test_set()
    model = crear_modelo()
    model.fit(train_data, train_y)
    guardar_modelo_svr(str(hour) + '00')
    print('Modelo guardado!')
    print('Error de validación: ' + str((model.predict(validation_data) - validation_y).std()))


Creando modelo para las 10...
Modelo guardado!
Error de validación: 0.05416437029788017

Creando modelo para las 11...
Modelo guardado!
Error de validación: 0.05898184865835802

Creando modelo para las 12...
Modelo guardado!
Error de validación: 0.06336144583763553

Creando modelo para las 13...
Modelo guardado!
Error de validación: 0.04371170439656865

Creando modelo para las 14...
Modelo guardado!
Error de validación: 0.043686322515764785

Creando modelo para las 15...
Modelo guardado!
Error de validación: 0.046305362629968094

Creando modelo para las 16...
Modelo guardado!
Error de validación: 0.06234688683208714

Creando modelo para las 17...
Modelo guardado!
Error de validación: 0.053667250027135065

Creando modelo para las 18...
Modelo guardado!
Error de validación: 0.049804101853045124

Creando modelo para las 19...
Modelo guardado!
Error de validación: 0.04899598024343952

Creando modelo para las 20...
Modelo guardado!
Error de validación: 0.04791194047917602

Creando modelo p

## Visualizando los resultados

Cargamos las librerias de [Plotly](https://plot.ly/python/)

In [20]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

Seleccionamos los días que queremos mostrar

In [21]:
dias = df_orig.loc[df_orig.index.isin(validation_y.index), 'Dia']

Cargamos el modelo que queremos analizar:

In [22]:
hour = 10

# Cargamos el modelo
loaded_model = joblib.load('models/all_data_model_{}00.sav'.format(str(hour)))

# Creamos los datos de validación
df = df_orig.copy()
eliminar_columnas(str(hour) + ':00')
train_data, test_data, validation_data, train_y, test_y, validation_y = training_test_set()

Visualizamos

In [23]:
iplot({
    'data': [go.Scatter(
                x=dias,
                y=validation_y * max_value,
                name='Real'
            ),go.Scatter(
                x=dias,
                y=loaded_model.predict(validation_data) * max_value,
                name='Predicción'
    )],
    'layout': go.Layout(
                yaxis={
                    'title': 'Ventas €'
                }
    )
})